In [3]:
# !pip install pyspark

     |████████████████████████████████| 204.7MB 65kB/s 
     |████████████████████████████████| 204kB 45.5MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.0-py2.py3-none-any.whl size=205044182 sha256=b526b865dc22399ff1c313714ec7ee847cb52902df0c84a0570454c10a6e7c92
  Stored in directory: /root/.cache/pip/wheels/57/27/4d/ddacf7143f8d5b76c45c61ee2e43d9f8492fc5a8e78ebd7d37
Successfully built pyspark


# **# Requisitos**

***1.   Conversão do formato dos arquivos: Converter o arquivo CSV presente no diretório data/input/users/load.csv, para um formato colunar de alta performance de leitura de sua escolha. Justificar brevemente a escolha do formato;***

**Justificativa da escolha do formato PARQUET**

Além das vantagens em utilizar uma representação colunar, o formato Parquet possui as seguintes vantagens/funcionalidades:

*   Estruturas de Dados Complexas
*   Compactação e Enconding
*   Null Values
*   Filter Pushdown

In [47]:
from pyspark.sql import SparkSession
import json
from pyspark.sql.functions import col, rank, max,  dense_rank
from pyspark.sql.window import Window

spark = SparkSession.builder.appName("CognitivoAI").getOrCreate()

In [12]:
# lendo arquivo load.csv da CognitivoAI
df = spark.read.csv("sample_data/load.csv", inferSchema=True, header=True)

In [13]:
#df.show()

+---+--------------------+--------------------+---------------+--------------------+---+--------------------+--------------------+
| id|                name|               email|          phone|             address|age|         create_date|         update_date|
+---+--------------------+--------------------+---------------+--------------------+---+--------------------+--------------------+
|  1|david.lynch@cogni...|         David Lynch|(11) 99999-9997|Mulholland Drive,...| 72|2018-03-03 18:47:...|2018-03-03 18:47:...|
|  1|david.lynch@cogni...|         David Lynch|(11) 99999-9998|Mulholland Drive,...| 72|2018-03-03 18:47:...|2018-04-14 17:09:...|
|  2|sherlock.holmes@c...|     Sherlock Holmes|(11) 94815-1623|221B Baker Street...| 34|2018-04-21 20:21:...|2018-04-21 20:21:...|
|  3|spongebob.squarep...|Spongebob Squarep...|(11) 91234-5678|124 Conch Street,...| 13|2018-05-19 04:07:...|2018-05-19 04:07:...|
|  1|david.lynch@cogni...|         David Lynch|(11) 99999-9999|Mulholland Drive,...

In [14]:
# Gravação do Arquivo no formato PARQUET
df.write.format("parquet").save("output_parquet")

In [15]:
# teste para ler o parquet
df_parquet = spark.read.parquet("output_parquet")

In [16]:
#df_parquet.show()

+---+--------------------+--------------------+---------------+--------------------+---+--------------------+--------------------+
| id|                name|               email|          phone|             address|age|         create_date|         update_date|
+---+--------------------+--------------------+---------------+--------------------+---+--------------------+--------------------+
|  1|david.lynch@cogni...|         David Lynch|(11) 99999-9997|Mulholland Drive,...| 72|2018-03-03 18:47:...|2018-03-03 18:47:...|
|  1|david.lynch@cogni...|         David Lynch|(11) 99999-9998|Mulholland Drive,...| 72|2018-03-03 18:47:...|2018-04-14 17:09:...|
|  2|sherlock.holmes@c...|     Sherlock Holmes|(11) 94815-1623|221B Baker Street...| 34|2018-04-21 20:21:...|2018-04-21 20:21:...|
|  3|spongebob.squarep...|Spongebob Squarep...|(11) 91234-5678|124 Conch Street,...| 13|2018-05-19 04:07:...|2018-05-19 04:07:...|
|  1|david.lynch@cogni...|         David Lynch|(11) 99999-9999|Mulholland Drive,...

In [20]:
#!ls sample_data -lrt

total 55512
-rwxr-xr-x 1 root root      930 Jan  1  2000 README.md
-rwxr-xr-x 1 root root     1697 Jan  1  2000 anscombe.json
-rw-r--r-- 1 root root  1706430 Jun 17 16:18 california_housing_train.csv
-rw-r--r-- 1 root root   301141 Jun 17 16:18 california_housing_test.csv
-rw-r--r-- 1 root root 36523880 Jun 17 16:18 mnist_train_small.csv
-rw-r--r-- 1 root root 18289443 Jun 17 16:18 mnist_test.csv
-rw-r--r-- 1 root root     1025 Jun 23 20:52 load.csv
drwxr-xr-x 2 root root     4096 Jun 23 20:54 input-parquet


In [21]:
#!ls -lrt output_parquet/

total 4
-rw-r--r-- 1 root root 2998 Jun 23 21:05 part-00000-8c661b3d-b76f-47bd-9914-60e940097f0f-c000.snappy.parquet
-rw-r--r-- 1 root root    0 Jun 23 21:05 _SUCCESS


***2.   Deduplicação dos dados convertidos: No conjunto de dados convertidos haverão múltiplas entradas para um mesmo registro, variando apenas os valores de alguns dos campos entre elas. Será necessário realizar um processo de deduplicação destes dados, a fim de apenas manter a última entrada de cada registro, usando como referência o id para identificação dos registros duplicados e a data de atualização (update_date) para definição do registro mais recente;***

In [55]:
MyWindow = Window.partitionBy(df['id']).orderBy(df['update_date'].desc())

In [61]:
df2_Window = df.withColumn("rank",rank().over(MyWindow))
#df2_Window.show()

+---+--------------------+--------------------+---------------+--------------------+---+--------------------+--------------------+----+
| id|                name|               email|          phone|             address|age|         create_date|         update_date|rank|
+---+--------------------+--------------------+---------------+--------------------+---+--------------------+--------------------+----+
|  1|david.lynch@cogni...|         David Lynch|(11) 99999-9999|Mulholland Drive,...| 72|2018-03-03 18:47:...|2018-05-23 10:13:...|   1|
|  1|david.lynch@cogni...|         David Lynch|(11) 99999-9998|Mulholland Drive,...| 72|2018-03-03 18:47:...|2018-04-14 17:09:...|   2|
|  1|david.lynch@cogni...|         David Lynch|(11) 99999-9997|Mulholland Drive,...| 72|2018-03-03 18:47:...|2018-03-03 18:47:...|   3|
|  3|spongebob.squarep...|Spongebob Squarep...|(11) 98765-4321|122 Conch Street,...| 13|2018-05-19 04:07:...|2018-05-19 05:08:...|   1|
|  3|spongebob.squarep...|Spongebob Squarep...|(

In [62]:
df2_final_rank = df2_Window.filter("rank=1").drop("rank")
#df2_final_rank.show()

+---+--------------------+--------------------+---------------+--------------------+---+--------------------+--------------------+
| id|                name|               email|          phone|             address|age|         create_date|         update_date|
+---+--------------------+--------------------+---------------+--------------------+---+--------------------+--------------------+
|  1|david.lynch@cogni...|         David Lynch|(11) 99999-9999|Mulholland Drive,...| 72|2018-03-03 18:47:...|2018-05-23 10:13:...|
|  3|spongebob.squarep...|Spongebob Squarep...|(11) 98765-4321|122 Conch Street,...| 13|2018-05-19 04:07:...|2018-05-19 05:08:...|
|  2|sherlock.holmes@c...|     Sherlock Holmes|(11) 94815-1623|221B Baker Street...| 34|2018-04-21 20:21:...|2018-04-21 20:21:...|
+---+--------------------+--------------------+---------------+--------------------+---+--------------------+--------------------+



In [60]:
# Gravação do Arquivo no formato PARQUET
df2_final_rank.write.format("parquet").save("output_parquet_req2")

AnalysisException: ignored

**3. Conversão do tipo dos dados deduplicados: No diretório config haverá um arquivo JSON de configuração (types_mapping.json), contendo os nomes dos campos e os respectivos tipos desejados de output. Utilizando esse arquivo como input, realizar um processo de conversão dos tipos dos campos descritos, no conjunto de dados deduplicados;**


In [64]:
# tipagem atual
#df2_final_rank.dtypes

[('id', 'int'),
 ('name', 'string'),
 ('email', 'string'),
 ('phone', 'string'),
 ('address', 'string'),
 ('age', 'int'),
 ('create_date', 'string'),
 ('update_date', 'string')]

In [65]:
# usei o arquivo "types_mapping.json" no formato 
with open('sample_data/types_mapping.json') as json_file:  
	    dados = json.load(json_file)
	    for t in dados:
	      df2_final_rank = df2_final_rank.withColumn(t,col(t).cast(dados[t]))


In [66]:
#df2_final_rank.dtypes

[('id', 'int'),
 ('name', 'string'),
 ('email', 'string'),
 ('phone', 'string'),
 ('address', 'string'),
 ('age', 'int'),
 ('create_date', 'timestamp'),
 ('update_date', 'timestamp')]

In [67]:
# Gravação do Arquivo no formato PARQUET
df2_final_rank.write.format("parquet").save("output_parquet_req3")

# Notas gerais
- Todas as operações devem ser realizadas utilizando Spark. O serviço de execução fica a seu critério, podendo utilizar tanto serviços locais como serviços em cloud. Justificar brevemente o serviço escolhido (EMR, Glue, Zeppelin, etc.).

- Cada operação deve ser realizada no dataframe resultante do passo anterior, podendo ser persistido e carregado em diferentes conjuntos de arquivos após cada etapa ou executados em memória e apenas persistido após operação final.

- Você tem liberdade p/ seguir a sequência de execução desejada;

- Solicitamos a transformação de tipos de dados apenas de alguns campos. Os outros ficam a seu critério

- O arquivo ou o conjunto de arquivos finais devem ser compactados e enviados por e-mail.

@author: Vagner dos Santos Macedo